In [17]:
##############
#K. David Roell CFPB 
#collects row-level and counts of data that fail distribution style quality edts
#requires precomputed bins loaded as bin_df from ../linked_dist_stats/IQR_all.csv
#this file is generated using the IQR cleaning viz script which uses data from the get_linked_dist script
##############
from collections import OrderedDict
import json
import os
import pandas as pd
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from math import sqrt
get_ipython().magic(u'matplotlib inline')
matplotlib.style.use('ggplot')

with open('../quality_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
    edit_sql = json.load(f)
    
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',
}

try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)
    
#read csv of distributions
bin_df = pd.read_csv('../linked_dist_stats/IQR_all.csv')
bin_df.drop('Unnamed: 0', inplace=True, axis=1)
bin_df.tail()

i'm connected


,name,std,mean,median,max_mult,med_mult,min_mult,q98,q02,q75,q25,iqr,min_q,max_q,count of loans,pct of total loans
15,440 to 470,428.700445,744.273204,686.0,4.318182,1.511777,0.002268,1732.00,101.0,1000.0,396.0,604.0,-510.0,1906.0,38168,1.654849
16,470 to 500,469.873442,785.687732,709.0,4.468085,1.467230,0.002033,1910.00,100.0,1088.0,404.0,684.0,-622.0,2114.0,32368,1.403379
17,500 to 530,490.119587,815.166159,730.0,4.400000,1.427203,0.001961,1980.00,107.0,1130.0,415.0,715.0,-657.5,2202.5,29552,1.281285
18,530 to 560,519.733182,844.413133,750.0,4.450094,1.373407,0.017921,2030.94,104.0,1196.0,417.0,779.0,-751.5,2364.5,23776,1.030855
19,561 and up,1136.066534,1199.800013,960.0,128.533419,1.025201,0.000357,4000.00,105.0,1575.0,485.0,1090.0,-1150.0,3210.0,231810,10.050580


In [18]:
#get counts based on new distribution method
def get_count_edit_fails(table=None, inc_max=None, inc_min=None, amt_min=None, amt_max=None, name=None, prop_type="'1', '2', '3'", loan_purp="'1','2','3'", loan_type="'1','2','3','4'", occupancy="'1','2','3'", lien="'1','2','3'"):
    """generates a SQL statement to get counts of loans outside of expected distributions"""
    
    #set base selection and table
    base_SQL = """SELECT '{name}' AS bin, {inc_max} AS bin_top, COUNT(amount) FROM {table} """.format(table=table, name=name, inc_max=inc_max)
    
    #set conditions for edit failure
    where_SQL = """WHERE (amount::INT NOT BETWEEN {amt_min} AND {amt_max}) AND
    (income::INT BETWEEN {inc_min} AND {inc_max}) """.format(amt_min=amt_min, amt_max=amt_max, inc_min=inc_min, inc_max=inc_max)
    
    filter_SQL = """AND property_type IN ({prop_type}) AND loan_type IN ({loan_type}) AND 
    loan_purpose IN ({loan_purp}) AND occupancy IN ({occupancy}) AND lien IN ({lien}) 
    """.format(prop_type=prop_type, loan_type=loan_type, loan_purp=loan_purp, occupancy=occupancy, lien=lien)
    #remove NA values from numeric fields
    na_SQL = """AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;"""

    final_SQL = base_SQL + where_SQL + filter_SQL + na_SQL #combine SQL strings to form query

    return final_SQL
    #sum failed rows



In [19]:
#get rows of failed edits with bin label
def get_row_edit_fails(table=None, inc_max=None, inc_min=None, amt_min=None, amt_max=None, name=None, prop_type="'1', '2', '3'", loan_purp="'1','2','3'", loan_type="'1','2','3','4'", occupancy="'1','2','3'", lien="'1','2','3'"):
    """generates a SQL statement to get rows of loans outside of expected distributions"""
    
    #set base selection and table
    base_SQL = """SELECT '{name}' AS bin, {inc_max} AS bin_top, agency, RID, income, amount, sequence FROM {table} """.format(table=table,  name=name, inc_max=inc_max)
    
    #set conditions for edit failure
    where_SQL = """WHERE (amount::INT NOT BETWEEN {amt_min} AND {amt_max}) AND
    (income::INT BETWEEN {inc_min} AND {inc_max}) """.format(amt_min=amt_min, amt_max=amt_max, inc_min=inc_min, inc_max=inc_max)
    
    filter_SQL = """AND property_type IN ({prop_type}) AND loan_type IN ({loan_type}) AND 
    loan_purpose IN ({loan_purp}) AND occupancy IN ({occupancy}) AND lien IN ({lien}) 
    """.format(prop_type=prop_type, loan_type=loan_type, loan_purp=loan_purp, occupancy=occupancy, lien=lien)
    #remove NA values from numeric fields
    na_SQL = """AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;"""

    final_SQL = base_SQL + where_SQL + filter_SQL + na_SQL #combine SQL strings to form query

    return final_SQL
    #sum failed rows



In [20]:
#get edit fails based on distribution method
#this cell retrieves counts of edit fails by year (year must be adjusted manually)
#FIXME: add second query to get individual records that failed

#kwargs are used to set filters for market product
#this script should be tested with all loan types and property types


kwargs= {'table':'hmdalar2014',
         'prop_type':"'1'",
        'loan_purp':"'1'",
        'loan_type':"'1'",
        'occupancy':"'1'", 
        'lien':"'1'"}


first = True #set boolean flag to instantiate first instance of dataframe
#FIXME: convet this to a function that takes a function argument for get_count_edit_fails
for index, row in bin_df.iterrows(): #iterate over bins of income from bin_df 
#    print(index)
    #set variable arguments for sql query filters
    kwargs['name'] = bin_df.name.loc[index]

    
    kwargs['inc_min'] = int(kwargs['name'].rsplit(' ', 2)[0])
    if kwargs['name'][-2:] == 'up':
        kwargs['inc_max'] = 9999
    else:
        kwargs['inc_max'] = int(kwargs['name'].rsplit(' ', 2)[2])
    
    kwargs['amt_min'] = int(bin_df.q02.loc[index])
    kwargs['amt_max'] = int(bin_df.q98.loc[index])
    
    sql = get_count_edit_fails(**kwargs) #format query using kwargs dictionary
    print(sql) #check query text
    cur.execute(sql,) #execute SQL
    data_df = pd.DataFrame(cur.fetchall()) #convert results to dataframe
    
    col_names = [desc[0] for desc in cur.description] # get column names from SQL table    
    if len(data_df.columns) == len(col_names): #name dataframe columns if query not empty

        print('setting column names')
        data_df.columns = col_names

    if first == True and len(data_df.columns) > 0: #set master data frame
        dist_df = data_df.copy()
        first = False

    elif first == False and len(data_df.columns) > 0:
        print('merging dataframes')
        dist_df=pd.concat([dist_df, data_df]) #concat new data to master dataframe

    else:
        print('no data returned from query')

    path = '../linked_dist_stats/' #establish data file path
    if not os.path.exists(path):
        os.makedirs(path)
        
    print(dist_df.tail()) #check dataframe contens

#    dist_df['name'] = bin_df.name.loc[index] #set bin ranges as name
    dist_df.to_csv(path + 'distribution_edit_fails.csv') #save dataframe to file

SELECT '0 to 20' AS bin, 20 AS bin_top, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 7 AND 349) AND
    (income::INT BETWEEN 0 AND 20) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
       bin  bin_top  count
0  0 to 20       20    661
SELECT '20 to 50' AS bin, 50 AS bin_top, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 8 AND 450) AND
    (income::INT BETWEEN 20 AND 50) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
merging dataframes
        bin  bin_top  count
0   0 to 20       20    661
0  20 to 50       50   1180
SELECT '50 to 80' AS bin, 80 AS bin_top, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 14 AND 570) AND
    (i

In [21]:
#get rows of edit fails
#get edit fails based on distribution method
#this cell retrieves counts of edit fails by year (year must be adjusted manually)
#FIXME: add second query to get individual records that failed

#kwargs are used to set filters for market product
#this script should be tested with all loan types and property types
kwargs= {'table':'hmdalar2014',
         'prop_type':"'1'",
        'loan_purp':"'1'",
        'loan_type':"'1'",
        'occupancy':"'1'", 
        'lien':"'1'"}


first = True #set boolean flag to instantiate first instance of dataframe

for index, row in bin_df.iterrows(): #iterate over bins of income from bin_df 
#    print(index)
    #set variable arguments for sql query filters
    kwargs['name'] = bin_df.name.loc[index]
    kwargs['inc_min'] = int(kwargs['name'].rsplit(' ', 2)[0])
    if kwargs['name'][-2:] == 'up':
        kwargs['inc_max'] = 9999
    else:
        kwargs['inc_max'] = int(kwargs['name'].rsplit(' ', 2)[2])
    kwargs['amt_min'] = bin_df.q02.loc[index]
    kwargs['amt_max'] = bin_df.q98.loc[index]
    
    sql = get_row_edit_fails(**kwargs) #format query using kwargs dictionary
    print(sql) #check query text
    cur.execute(sql,) #execute SQL
    data_df = pd.DataFrame(cur.fetchall()) #convert results to dataframe
    
    col_names = [desc[0] for desc in cur.description] # get column names from SQL table
    
    if len(data_df.columns) == len(col_names): #name dataframe columns if query not empty
        print('setting column names')
        data_df.columns = col_names

    if first == True and len(data_df.columns) > 0: #set master data frame
        dist_df = data_df.copy()
        first = False

    elif first == False and len(data_df.columns) > 0:
        print('merging dataframes')
        dist_df=pd.concat([dist_df, data_df]) #concat new data to master dataframe

    else:
        print('no data returned from query')

    path = '../linked_dist_stats/' #establish data file path
    if not os.path.exists(path):
        os.makedirs(path)
        
    print(dist_df.tail()) #check dataframe contens

    dist_df['name'] = bin_df.name.loc[index] #set bin ranges as name
    dist_df.to_csv(path + 'distribution_row_edit_fails.csv') #save dataframe to file

SELECT '0 to 20' AS bin, 20 AS bin_top, agency, RID, income, amount, sequence FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 7.0 AND 349.0) AND
    (income::INT BETWEEN 0 AND 20) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
         bin  bin_top agency         rid income amount sequence
656  0 to 20       20      5  0000011065   0001  00529  0003478
657  0 to 20       20      3  0000019899   0003  00417  0000548
658  0 to 20       20      9  0000497404   0001  01100  0023529
659  0 to 20       20      9  0002489805   0002  01500  0003408
660  0 to 20       20      9  0000451965   0001  02500  0422997
SELECT '20 to 50' AS bin, 50 AS bin_top, agency, RID, income, amount, sequence FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 8.0 AND 450.8199999999924) AND
    (income::INT BETWEEN 20 AND 50) AND property_type IN ('1') AND loan_

ValueError: invalid literal for int() with base 10: 'up'

In [ ]:
#compress tail bins for counts df into one 600+ income
fail_counts_df = pd.read_csv(path + 'distribution_edit_fails.csv')

In [ ]:
fail_counts_df.head()